# Custom Dense Layer

이번 튜토리얼에서는 Custom 한 Dense Layer 층을 만드는 방법을 배웁니다.

Layer는 `tf.keras.layers`의 `Layer`를 상속받아 Custom 한 Layer를 생성합니다.

## Imports

In [1]:
import tensorflow as tf
import numpy as np
from pprint import pprint

## Custom Layer with weights

`tf.keras.layers`의 `Layer`를 상속받아 만들되, 이번에는 `build()` 메서드도 활용합니다.

`build()` 메서드에서 필요한 가중치를 초기화할 수 있습니다.

In [2]:
from tensorflow.keras.layers import Layer

class MyDense(Layer):

    def __init__(self, units=32):
        super(MyDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''가중치를 초기화 합니다.'''
        # weight를 초기화 합니다.
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="weight", 
                             initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                             trainable=True)

        # bias를 초기화 합니다.
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias", 
                             initial_value=b_init(shape=(self.units,), dtype='float32'),
                             trainable=True)

    def call(self, inputs):
        '''wx + b'''
        return tf.matmul(inputs, self.w) + self.b

단순 선형 회귀를 위의 custom layer로 풀어낼 수 있습니다.

In [3]:
# Dense 
mydense = MyDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = mydense(x)

# parameters of the base Layer class like `variables` can be used
print(mydense.variables)

[<tf.Variable 'my_dense/weight:0' shape=(1, 1) dtype=float32, numpy=array([[0.09741727]], dtype=float32)>, <tf.Variable 'my_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


## 선형회귀 모델 생성 및 훈련

In [4]:
# 샘플 데이터셋 y = 2x -1
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# Custom Layer를 활용한 모델 생성
dense = MyDense(units=1)
model = tf.keras.Sequential([dense])

# train model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

# make predictions
pprint(model.predict([10.0]))

# check weights
pprint(dense.weights)

array([[18.981771]], dtype=float32)
[<tf.Variable 'my_dense_1/weight:0' shape=(1, 1) dtype=float32, numpy=array([[1.997358]], dtype=float32)>,
 <tf.Variable 'my_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9918088], dtype=float32)>]


## Custom Layer에 activation 을 Hyperparameter 지정

`'relu'` 지정으로 activation을 가져올 수 있습니다.

In [5]:
tf.keras.activations.get('relu')

<function tensorflow.python.keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0)>

In [6]:
tf.keras.activations.get('gelu')

<function tensorflow.python.keras.activations.gelu(x, approximate=False)>

None을 지정시 default로 `linear activation` 적용

In [7]:
tf.keras.activations.get(None)

<function tensorflow.python.keras.activations.linear(x)>

## Custom Layer에 적용

In [8]:
from tensorflow.keras.layers import Layer

class MyDense(Layer):

    def __init__(self, units=32, activation=None):
        super(MyDense, self).__init__()
        self.units = units
        
        # Keras의 built-in activation을 가져옵니다.
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="weight", 
                             initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                             trainable=True)

        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias", 
                             initial_value=b_init(shape=(self.units,), dtype='float32'),
                             trainable=True)

    def call(self, inputs):
        '''wx + b'''
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [9]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    # custom layer 삽입
    MyDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4876 - acc: 0.8592
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1518 - acc: 0.9556
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1069 - acc: 0.9682
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0895 - acc: 0.9724
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0707 - acc: 0.9792


[0.07071426510810852, 0.979200005531311]